In [2]:
import numpy as np
import cv2
import pandas as pd
import os
import pickle
from google.colab.patches import cv2_imshow

In [ ]:
# mounting drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path = '/content/drive/MyDrive/img_cv/'
os.listdir(path)

In [5]:
# face detection model
face_detection_model = path + 'res10_300x300_ssd_iter_140000_fp16.caffemodel'

# mô tả kiến trúc
face_detection_proto = path + 'deploy.prototxt'

# mô hình mô tả khuôn mặt
face_descriptor = path + 'openface.nn4.small2.v1.t7'

# load model dnn nhận diện khuôn mặt
face_detector = cv2.dnn.readNetFromCaffe(face_detection_proto, face_detection_model)

# đọc mô tả khuôn mặt file Torch
descriptor_model = cv2.dnn.readNetFromTorch(face_descriptor)


In [ ]:
# danh sách hình ảnh đầu vào
faces = '/content/drive/MyDrive/img_cv/test_img/hochiminh/'
os.listdir(faces)

# throuh the folder to show img
for img in os.listdir(faces):
    # Construct the full image path
    image_path = os.path.join(faces, img)

    # Read the image using cv2.imread()
    image = cv2.imread(image_path)

    # Display the image
    cv2_imshow(image)

In [ ]:
# danh sách hình ảnh đầu vào
path = '/content/drive/MyDrive/img_cv/test_img/hochiminh/hochiminh_2.jpg'
img = cv2.imread(path)

# lay ban copy cua anh
image = img.copy()

#lay du lieu chieu cao va chieu rong cua anh
h,w = image.shape[:2]

#chuan bi du lieu dau vao cho mo hinh nhan dien khuon mat
imgBlob = cv2.dnn.blobFromImage(image, 1, (300,300), (104,177,123), swapRB=False, crop=False)

# set du lieu dau vao cho mo hinh
face_detector.setInput(imgBlob)

# nhan dien khuon mat
detections = face_detector.forward()

# kiem tra xem co khuon mat nao khong
if detections.any():
    # get the highest confidence face
    i = np.argmax(detections[0,0,:,2])
    confidence = detections[0,0,i,2]
    if confidence > 0.5:
        # get point to draw
        box = detections[0,0,i,3:7] * np.array([w,h,w,h])
        (x1,y1,x2,y2) = box.astype('int')
        # crop khung anh de trich xuat khuon mat
        face_img = image[y1:y2, x1:x2].copy()
        cv2_imshow(face_img)

        # chuẩn bị dữ liệu cho đầu vào mô hình trích xuất đặc trưng
        face_blob = cv2.dnn.blobFromImage(face_img, 1/255, (96,96), (0,0,0), swapRB=True, crop=True)
        # thiết lập đầu vào cho model
        descriptor_model.setInput(face_blob)
        # trích xuất đặc trưng
        vectors = descriptor_model.forward()
        print(vectors)

In [10]:
# tạo function để áp dụng cho nhiều ảnh
def detector(img_path):
  img = cv2.imread(img_path)

  # lay ban copy cua anh
  image = img.copy()

  #lay du lieu chieu cao va chieu rong cua anh
  h,w = image.shape[:2]

  #chuan bi du lieu dau vao cho mo hinh nhan dien khuon mat
  imgBlob = cv2.dnn.blobFromImage(image, 1, (300,300), (104,177,123), swapRB=False, crop=False)

  # set du lieu dau vao cho mo hinh
  face_detector.setInput(imgBlob)

  # nhan dien khuon mat
  detections = face_detector.forward()

  # kiem tra xem co khuon mat nao khong
  if detections.any():
      # get the highest confidence face
      i = np.argmax(detections[0,0,:,2])
      confidence = detections[0,0,i,2]
      if confidence > 0.5:
          # get point to draw
          box = detections[0,0,i,3:7] * np.array([w,h,w,h])
          (x1,y1,x2,y2) = box.astype('int')
          # crop khung anh de trich xuat khuon mat
          face_img = image[y1:y2, x1:x2].copy()
          cv2_imshow(face_img)

          # chuẩn bị dữ liệu cho đầu vào mô hình trích xuất đặc trưng
          face_blob = cv2.dnn.blobFromImage(face_img, 1/255, (96,96), (0,0,0), swapRB=True, crop=True)
          # thiết lập đầu vào cho model
          descriptor_model.setInput(face_blob)
          # trích xuất đặc trưng
          vectors = descriptor_model.forward()
          return vectors

In [ ]:
# apply function cho toan bo image
# danh sách hình ảnh đầu vào
faces = '/content/drive/MyDrive/img_cv/test_img/hochiminh/'
folder = faces.split('/')[-2]

# khoi tao dictionary
data = dict(data=[], label=[])


# throuh the folder to show img
for img in os.listdir(faces):
  try:
    # Construct the full image path
    image_path = os.path.join(faces, img)

    # detect cac image
    vector = detector(image_path)
    # kiem tra viec trich xuat dac trung neu thanh cong
    if vector is not None:
      # them vector dac trung vao tu dien voi label la ten thu muc
      data['data'].append(vector)
      data['label'].append(folder)
      print('Success')
  except:
    # skip neu loi
    print('Skip')
    continue


In [ ]:
data.keys()

In [ ]:
# tao 1 series tu dictionary
labeled_series = pd.Series(data['label'], name='Label')
# dem so lan xuat ien cua moi label
labeled_series.value_counts()